In [50]:
import pandas as pd
import torch
import dgl
import networkx as nx
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from dgl.nn import GraphConv
import time
import random


In [51]:
nodes_df = pd.read_csv('clean_nodes.csv')
edges_df = pd.read_csv('clean_edges.csv')

1.Xây dựng đồ thị
# Tạo mapping từ profile_id sang index (0 đến N-1)
# Tạo tensor cho edge_index (dạng (2, num_edges))
# Tạo đồ thị DGL

In [52]:
profile_ids = nodes_df['profile_id'].tolist()
node_to_idx = {user: i for i, user in enumerate(profile_ids)}

# Xây dựng danh sách cạnh (chỉ thêm các cạnh có cả source và target nằm trong nodes)
edge_list = []
for idx, row in edges_df.iterrows():
    src, tgt = row['source'], row['target']
    if src in node_to_idx and tgt in node_to_idx:
        edge_list.append( (node_to_idx[src], node_to_idx[tgt]) )

if len(edge_list) == 0:
    raise ValueError("Không tìm thấy cạnh hợp lệ!")

In [53]:
src_nodes, tgt_nodes = zip(*edge_list)
edge_index_tensor = torch.tensor([src_nodes, tgt_nodes], dtype=torch.int64)

g = dgl.graph((edge_index_tensor[0], edge_index_tensor[1]), num_nodes=len(profile_ids))
print("DGL Graph: {} nút, {} cạnh".format(g.num_nodes(), g.num_edges()))

DGL Graph: 25669 nút, 53452 cạnh


2. Tính toán các chỉ số mạng cơ bản

In [ ]:
nx_g = dgl.to_networkx(g)
print("Đã chuyển sang NetworkX.")

# Nếu đồ thị lớn (n > 1000), dùng xấp xỉ cho betweenness (sample k nodes)
num_nodes = g.num_nodes()
k_sample = 100 if num_nodes > 1000 else None
in_degrees = nx.in_degree_centrality(nx_g)
out_degrees = nx.out_degree_centrality(nx_g)
start_time = time.time()
if k_sample:
    betweenness = nx.betweenness_centrality(nx_g, k=k_sample, normalized=True)
else:
    betweenness = nx.betweenness_centrality(nx_g, normalized=True)
print("Betweenness Centrality tính xong trong {:.2f} giây.".format(time.time()-start_time))

start_time = time.time()
# Closeness có thể tính trực tiếp (cẩn thận với đồ thị lớn)
closeness = nx.closeness_centrality(nx_g)
print("Closeness Centrality tính xong trong {:.2f} giây.".format(time.time()-start_time))

start_time = time.time()
# Eigenvector Centrality: nếu không hội tụ, bắt ngoại lệ và gán 0 cho nút đó
try:
    eigenvector = nx.eigenvector_centrality(nx_g)
except Exception as e:
    print("Eigenvector Centrality không hội tụ:", e)
    eigenvector = {i: 0.0 for i in range(num_nodes)}
print("Eigenvector Centrality tính xong trong {:.2f} giây.".format(time.time()-start_time))

# Tạo DataFrame chứa các chỉ số mạng (sắp xếp theo index của node)
metrics_df = pd.DataFrame({
    'profile_id': profile_ids,
    'in_degree': [in_degrees.get(node, 0) for node in range(num_nodes)],
    'out_degree': [out_degrees.get(node, 0) for node in range(num_nodes)],
    'betweenness': [betweenness.get(i, 0.0) for i in range(num_nodes)],
    'closeness': [closeness.get(i, 0.0) for i in range(num_nodes)],
    'eigenvector': [eigenvector.get(i, 0.0) for i in range(num_nodes)]
})
print("Các chỉ số mạng tính được, hiển thị 5 dòng đầu:")
print(metrics_df.head())

Đã chuyển sang NetworkX.
Betweenness Centrality tính xong trong 11.87 giây.
Closeness Centrality tính xong trong 195.07 giây.
Eigenvector Centrality không hội tụ: not implemented for multigraph type
Eigenvector Centrality tính xong trong 0.06 giây.
Các chỉ số mạng tính được, hiển thị 5 dòng đầu:
   profile_id  in_degree  out_degree  betweenness  closeness  eigenvector
0          14         61          54     0.010998   0.191469          0.0
1         512          0           1     0.000000   0.000000          0.0
2         533          1           1     0.000000   0.183487          0.0
3         534       3252        2825     0.170321   0.254787          0.0
4         564        110           3     0.000269   0.194558          0.0


In [55]:
metrics_df[metrics_df['in_degree'] > 0]

,profile_id,in_degree,out_degree,betweenness,closeness,eigenvector
0,14,61,54,0.010998,0.191469,0.0
2,533,1,1,0.000000,0.183487,0.0
3,534,3252,2825,0.170321,0.254787,0.0
4,564,110,3,0.000269,0.194558,0.0
5,598,2,3,0.000180,0.174467,0.0
...,...,...,...,...,...,...
25664,11313784,1,0,0.000000,0.129857,0.0
25665,11313915,1,0,0.000000,0.134640,0.0
25666,11313924,1,0,0.000000,0.134640,0.0
25667,11313953,1,0,0.000000,0.134640,0.0


# 3. Nhúng đồ thị sử dụng GNN với DGL (GCN)


In [56]:
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_feats)
        self.conv2 = GraphConv(hidden_feats, out_feats)
    
    def forward(self, graph, features):
        x = self.conv1(graph, features)
        x = F.relu(x)
        x = self.conv2(graph, x)
        return x

In [57]:
# Nếu chưa có đặc trưng cho nút, tạo feature ngẫu nhiên (ví dụ, kích thước 16)
feat_dim = 16
if 'feat' not in g.ndata:
    g.ndata['feat'] = torch.randn(g.num_nodes(), feat_dim)

embedding_dim = 16  # Kích thước vector nhúng mong muốn
model = GCN(feat_dim, 2 * embedding_dim, embedding_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)


In [58]:
# Để giảm chi phí tính toán, ta huấn luyện số epoch thấp (ví dụ 100 epoch)
num_epochs = 100
print("Kích thước z:", z.shape)
print("Kích thước g.ndata['feat']:", g.ndata['feat'].shape)

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    g = dgl.add_self_loop(g)
    # Ở đây, làm bài toán tự giám sát đơn giản: tái tạo lại đặc trưng ban đầu bằng MSE
    z = model(g, g.ndata['feat'])
    loss = F.mse_loss(z, g.ndata['feat'])
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:03d}: Loss = {loss.item():.4f}")

# Sau huấn luyện, trích xuất vector nhúng của các nút
model.eval()
with torch.no_grad():
    node_embeddings = model(g, g.ndata['feat']).cpu().numpy()

# Tạo DataFrame cho embedding: đặt tên cột là gnn_emb_0, gnn_emb_1, ... gnn_emb_63
embedding_df = pd.DataFrame(node_embeddings, columns=[f'gnn_emb_{i}' for i in range(embedding_dim)])
embedding_df['profile_id'] = profile_ids

Kích thước z: torch.Size([565841, 16])
Kích thước g.ndata['feat']: torch.Size([25669, 16])
Epoch 010: Loss = 0.5747
Epoch 020: Loss = 0.2677
Epoch 030: Loss = 0.1630
Epoch 040: Loss = 0.1233
Epoch 050: Loss = 0.0884
Epoch 060: Loss = 0.0662
Epoch 070: Loss = 0.0499
Epoch 080: Loss = 0.0382
Epoch 090: Loss = 0.0298
Epoch 100: Loss = 0.0237


In [59]:
##############################################
# 4. Kết hợp đặc trưng: Metrics + GNN Embedding
##############################################

combined_df = pd.merge(metrics_df, embedding_df, on='profile_id')
print("Kết quả đặc trưng kết hợp, 5 dòng đầu:")
print(combined_df.head())

# Lưu kết quả ra file CSV
combined_df.to_csv('combined_user_features.csv', index=False)
print("Đã lưu đặc trưng kết hợp vào file 'combined_user_features.csv'")

Kết quả đặc trưng kết hợp, 5 dòng đầu:
   profile_id  in_degree  out_degree  betweenness  closeness  eigenvector  \
0          14         61          54     0.010998   0.191469          0.0   
1         512          0           1     0.000000   0.000000          0.0   
2         533          1           1     0.000000   0.183487          0.0   
3         534       3252        2825     0.170321   0.254787          0.0   
4         564        110           3     0.000269   0.194558          0.0   

   gnn_emb_0  gnn_emb_1  gnn_emb_2  gnn_emb_3  ...  gnn_emb_6  gnn_emb_7  \
0   0.211611  -0.269107   0.273392  -0.296295  ...  -0.035242   0.167099   
1  -0.331892   0.245718  -0.127536   1.064909  ...  -0.488813   2.731494   
2   0.500691   0.150737  -0.137665  -1.646185  ...   2.055947  -1.145390   
3  -0.007179   0.653304  -0.476211   0.049931  ...  -0.598944   0.423170   
4  -0.534705   1.021134  -0.330630  -0.251206  ...  -1.197644  -0.229253   

   gnn_emb_8  gnn_emb_9  gnn_emb_10  gnn_